# Forced-Dissipative Turbulence in pygSQuiG

This notebook demonstrates:
1. Setting up ring forcing at intermediate scales
2. Adding large-scale damping
3. Reaching statistical steady state
4. Analyzing energy cascades
5. Computing spectral slopes and fluxes

## 1. Setup and Imports

In [ ]:
import numpy as np
import jax
import jax.numpy as jnp
import matplotlib.pyplot as plt
from matplotlib import cm

# pygSQuiG imports
from pygsquig.core.grid import make_grid, ifft2
from pygsquig.core.solver import gSQGSolver
from pygsquig.forcing.ring_forcing import RingForcing
from pygsquig.forcing.damping import CombinedDamping
from pygsquig.utils.diagnostics import (
    compute_energy_spectrum,
    compute_enstrophy,
    compute_total_energy,
)

# Set random seed for reproducibility
np.random.seed(42)
print("pygSQuiG forced turbulence simulation")

## 2. Configuration

For forced turbulence, we need:
- **Forcing**: Energy injection at intermediate scales
- **Small-scale dissipation**: Hyperviscosity to dissipate enstrophy
- **Large-scale damping**: Linear drag to dissipate energy

In [ ]:
# Grid parameters
N = 256          # Higher resolution for better cascade
L = 2 * np.pi    # Domain size

# Create grid
grid = make_grid(N, L)
print(f"Grid: {N}×{N} points, L={L:.2f}")

# Physical parameters
alpha = 1.0      # SQG turbulence
nu_p = 1e-16     # Small-scale hyperviscosity
p = 8            # Hyperviscosity order

# Forcing parameters
kf = 30.0        # Forcing wavenumber (intermediate scale)
dk = 2.0         # Forcing bandwidth
epsilon = 0.1    # Target energy injection rate

# Damping parameters
mu = 0.1         # Large-scale damping coefficient

print(f"\nPhysics configuration:")
print(f"  α = {alpha} (SQG)")
print(f"  Forcing at k_f = {kf} ± {dk}")
print(f"  Energy injection rate ε = {epsilon}")
print(f"  Large-scale damping μ = {mu}")
print(f"  Small-scale dissipation ν_{p} = {nu_p:.1e}")

## 3. Create Solver and Forcing

In [ ]:
# Create solver
solver = gSQGSolver(grid, alpha=alpha, nu_p=nu_p, p=p)

# Create forcing
forcing = RingForcing(kf=kf, dk=dk, epsilon=epsilon, tau_f=0.0)  # White noise forcing

# Create combined damping (large + small scales)
damping = CombinedDamping(mu=mu, kf=kf, nu_p=nu_p, p=p)

# Combined forcing function that includes both forcing and damping
def combined_forcing(theta_hat, **kwargs):
    """Apply forcing and damping together."""
    # Get forcing contribution
    if 'key' in kwargs and 'dt' in kwargs:
        forcing_hat = forcing(theta_hat, kwargs['key'], kwargs['dt'], grid)
    else:
        forcing_hat = jnp.zeros_like(theta_hat)
    
    # Get damping contribution
    damping_hat = damping(theta_hat, grid)
    
    # Combine
    return forcing_hat + damping_hat

print("Solver and forcing created successfully!")

## 4. Spin-up Phase

Start from low energy and let the system reach statistical steady state:

In [ ]:
# Initialize from low energy state
state = solver.initialize(seed=123)

# Time stepping parameters
dt = 0.001
n_spinup = 2000   # Spin-up steps

# Initialize random key for forcing
rng_key = jax.random.PRNGKey(456)

# Storage for monitoring
spinup_times = []
spinup_energies = []

print("Starting spin-up phase...")
for step in range(n_spinup):
    # Split key for this step
    rng_key, subkey = jax.random.split(rng_key)
    
    # Step forward
    state = solver.step(state, dt, forcing=combined_forcing, key=subkey, dt=dt)
    
    # Monitor every 100 steps
    if (step + 1) % 100 == 0:
        E = compute_total_energy(state['theta_hat'], grid, alpha)
        spinup_times.append(state['time'])
        spinup_energies.append(float(E))
        
        if (step + 1) % 500 == 0:
            print(f"  Step {step+1}: t={state['time']:.2f}, E={E:.4f}")

print(f"\nSpin-up complete! Final energy: {spinup_energies[-1]:.4f}")

In [ ]:
# Plot spin-up evolution
plt.figure(figsize=(8, 5))
plt.plot(spinup_times, spinup_energies, 'b-', linewidth=2)
plt.xlabel('Time')
plt.ylabel('Total Energy')
plt.title('Energy Evolution During Spin-up')
plt.grid(True, alpha=0.3)
plt.show()

# Check if steady state is reached
energy_change = abs(spinup_energies[-1] - spinup_energies[-5]) / spinup_energies[-1]
print(f"Relative energy change in last 500 steps: {energy_change*100:.2f}%")
if energy_change < 0.05:
    print("✓ System appears to be in statistical steady state")
else:
    print("⚠️ System may need more spin-up time")

## 5. Statistical Sampling

Now collect statistics in the steady state:

In [ ]:
# Parameters for statistics collection
n_stats = 1000        # Number of steps for statistics
sample_interval = 50  # Sample every N steps

# Storage
spectra = []
energies = []
enstrophies = []
injection_rates = []

print("Collecting statistics...")
for step in range(n_stats):
    # Split key
    rng_key, subkey = jax.random.split(rng_key)
    
    # Step forward
    state = solver.step(state, dt, forcing=combined_forcing, key=subkey, dt=dt)
    
    # Sample statistics
    if (step + 1) % sample_interval == 0:
        # Energy and enstrophy
        E = compute_total_energy(state['theta_hat'], grid, alpha)
        Z = compute_enstrophy(state['theta_hat'], grid, alpha)
        energies.append(float(E))
        enstrophies.append(float(Z))
        
        # Spectrum
        k_bins, E_k = compute_energy_spectrum(state['theta_hat'], grid, alpha)
        spectra.append(E_k)
        
        # Estimate injection rate
        forcing_hat = forcing(state['theta_hat'], subkey, dt, grid)
        theta = ifft2(state['theta_hat']).real
        forcing_phys = ifft2(forcing_hat).real
        injection = float(jnp.mean(theta * forcing_phys))
        injection_rates.append(injection)

print(f"\nStatistics collected over {n_stats*dt:.1f} time units")
print(f"Number of samples: {len(spectra)}")

## 6. Energy Balance Analysis

In [ ]:
# Compute mean values
mean_energy = np.mean(energies)
std_energy = np.std(energies)
mean_injection = np.mean(injection_rates)
mean_enstrophy = np.mean(enstrophies)

print("Energy Balance Analysis:")
print(f"  Mean energy: {mean_energy:.4f} ± {std_energy:.4f}")
print(f"  Mean enstrophy: {mean_enstrophy:.2e}")
print(f"  Mean injection rate: {mean_injection:.4f}")
print(f"  Target injection rate: {epsilon}")
print(f"  Relative error: {abs(mean_injection - epsilon)/epsilon*100:.1f}%")

# Plot time series
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8))

# Energy
ax1.plot(energies, 'b-', alpha=0.7)
ax1.axhline(mean_energy, color='r', linestyle='--', label=f'Mean: {mean_energy:.3f}')
ax1.set_ylabel('Energy')
ax1.set_title('Energy Time Series in Steady State')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Injection rate
ax2.plot(injection_rates, 'g-', alpha=0.7)
ax2.axhline(epsilon, color='r', linestyle='--', label=f'Target: {epsilon}')
ax2.axhline(mean_injection, color='k', linestyle=':', label=f'Mean: {mean_injection:.3f}')
ax2.set_xlabel('Sample')
ax2.set_ylabel('Injection Rate')
ax2.set_title('Energy Injection Rate')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 7. Energy Spectrum and Cascades

In [ ]:
# Compute mean spectrum
mean_spectrum = np.mean(spectra, axis=0)
std_spectrum = np.std(spectra, axis=0)

# Visualize θ field
theta = ifft2(state['theta_hat']).real

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# θ field
im = ax1.imshow(theta, cmap='RdBu_r', origin='lower',
                extent=[0, L, 0, L])
ax1.set_xlabel('x')
ax1.set_ylabel('y')
ax1.set_title('Buoyancy Field θ')
plt.colorbar(im, ax=ax1)

# Energy spectrum
ax2.loglog(k_bins, mean_spectrum, 'b-', linewidth=2, label='Mean spectrum')
ax2.fill_between(k_bins, mean_spectrum - std_spectrum, mean_spectrum + std_spectrum,
                 alpha=0.3, color='blue')

# Mark forcing scale
ax2.axvline(kf, color='g', linestyle='--', alpha=0.5, label=f'Forcing (k={kf:.0f})')
ax2.axvspan(kf-dk, kf+dk, alpha=0.2, color='g')

# Reference slopes
# Inverse cascade (k < kf): k^{-5/3}
k_inverse = k_bins[(k_bins > 5) & (k_bins < kf*0.8)]
if len(k_inverse) > 2:
    E_ref_inv = mean_spectrum[k_bins == k_inverse[-1]][0] * (k_inverse/k_inverse[-1])**(-5/3)
    ax2.loglog(k_inverse, E_ref_inv, 'r--', alpha=0.7, linewidth=1.5, label='k⁻⁵/³ (inverse)')

# Forward cascade (k > kf): k^{-3}
k_forward = k_bins[(k_bins > kf*1.5) & (k_bins < N/4)]
if len(k_forward) > 2:
    E_ref_fwd = mean_spectrum[k_bins == k_forward[0]][0] * (k_forward/k_forward[0])**(-3)
    ax2.loglog(k_forward, E_ref_fwd, 'k--', alpha=0.7, linewidth=1.5, label='k⁻³ (forward)')

ax2.set_xlabel('Wavenumber k')
ax2.set_ylabel('Energy Spectrum E(k)')
ax2.set_title('Energy Spectrum with Dual Cascade')
ax2.grid(True, alpha=0.3, which='both')
ax2.legend()
ax2.set_xlim(1, N/2)
ax2.set_ylim(1e-12, 1e0)

plt.tight_layout()
plt.show()

print("Cascade analysis:")
print(f"  Forcing scale: k_f = {kf}")
print(f"  Inverse cascade (k < k_f): energy flows to large scales with E(k) ~ k⁻⁵/³")
print(f"  Forward cascade (k > k_f): enstrophy flows to small scales with E(k) ~ k⁻³")

## 8. Parameter Sensitivity Study

In [ ]:
# Test different damping coefficients
mu_values = [0.05, 0.1, 0.2]
colors = ['blue', 'red', 'green']

fig, ax = plt.subplots(1, 1, figsize=(8, 6))

for mu_test, color in zip(mu_values, colors):
    # Create new damping
    damping_test = CombinedDamping(mu=mu_test, kf=kf, nu_p=nu_p, p=p)
    
    # Run for a bit with this damping
    state_test = solver.initialize(seed=789)
    
    # Quick spin-up
    for _ in range(500):
        rng_key, subkey = jax.random.split(rng_key)
        
        def test_forcing(theta_hat, **kwargs):
            forcing_hat = forcing(theta_hat, subkey, dt, grid)
            damping_hat = damping_test(theta_hat, grid)
            return forcing_hat + damping_hat
        
        state_test = solver.step(state_test, dt, forcing=test_forcing)
    
    # Compute spectrum
    k_test, E_test = compute_energy_spectrum(state_test['theta_hat'], grid, alpha)
    E_total = compute_total_energy(state_test['theta_hat'], grid, alpha)
    
    ax.loglog(k_test, E_test, color=color, linewidth=2, 
             label=f'μ = {mu_test}, E = {E_total:.2f}')

ax.axvline(kf, color='gray', linestyle='--', alpha=0.5)
ax.set_xlabel('Wavenumber k')
ax.set_ylabel('Energy Spectrum E(k)')
ax.set_title('Effect of Large-Scale Damping on Spectrum')
ax.grid(True, alpha=0.3, which='both')
ax.legend()
ax.set_xlim(1, N/2)

plt.show()

print("Observations:")
print("  - Larger μ suppresses large-scale energy more effectively")
print("  - The cascade slopes remain unchanged")
print("  - Total energy decreases with increasing μ")

## Summary

This notebook demonstrated:

1. **Forced-dissipative setup**: Ring forcing at intermediate scales with large-scale damping
2. **Statistical steady state**: System reaches equilibrium between injection and dissipation
3. **Dual cascade**:
   - **Inverse energy cascade** (k < k_f): E(k) ~ k^{-5/3}
   - **Forward enstrophy cascade** (k > k_f): E(k) ~ k^{-3}
4. **Energy balance**: Mean injection rate matches dissipation
5. **Parameter sensitivity**: Damping coefficient controls large-scale energy

### Key physics insights:
- SQG turbulence exhibits a dual cascade similar to 2D turbulence
- Energy flows upscale while enstrophy flows downscale
- The spectral slopes match theoretical predictions
- Proper balance of forcing and dissipation is crucial

### Next steps:
- Try different α values to see how cascades change
- Implement anisotropic forcing
- Add passive scalars to study mixing
- Use adaptive timestepping for efficiency